In [1]:
import boto3
import pandas as pd
from io import StringIO
import os
from dotenv import load_dotenv
import numpy as np
load_dotenv()

True

In [2]:
# Your AWS credentials and bucket info
bucket_name = 'my-feature-store-data'
s3_key = 'pipeline-data/data.csv'  # Example: "pipeline-data/data.csv"

# Create an S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id= os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
)

# Fetch the object from S3
response = s3.get_object(Bucket=bucket_name, Key=s3_key)

# Read the CSV content
csv_data = response['Body'].read().decode('utf-8')

# Convert to DataFrame
df = pd.read_csv(StringIO(csv_data))

# Done!
print(df.isnull().sum())

index                         0
aqi_index                     0
co                            0
no                            0
no2                           0
o3                            0
so2                           0
pm2_5                         0
pm10                          0
nh3                           0
temperature_2m                0
relative_humidity_2m          0
precipitation                 0
wind_speed_10m                0
wind_direction_10m            0
surface_pressure              0
dew_point_2m                  0
apparent_temperature          0
shortwave_radiation           0
et0_fao_evapotranspiration    0
year                          0
month                         0
day                           0
hour                          0
dtype: int64


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split


# Step 4: Define target and features
target = df["aqi_index"]
X = df.drop(columns=["aqi_index"])

# Step 5: Final check for datetime columns
X = X.select_dtypes(exclude=["datetime64[ns]"])

# Step 6: Split
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)


In [4]:
# Import necessary libraries
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import joblib

In [5]:
# ✅ Step 3: Define Models
models = {
    "Random_Forest": RandomForestRegressor(n_estimators=300, max_depth=10, random_state=42),
    "Gradient_Boosting": GradientBoostingRegressor(n_estimators=300, max_depth=3, random_state=42),
    "Linear_Regression": LinearRegression(),
    "Ridge_Regression": Ridge(alpha=1.0),
    "Support_Vector_Regressor": SVR(),
    "Neural_Network": MLPRegressor(max_iter=200, random_state=42)
}

# -----------------------
# ✅ Step 4: Train and Evaluate
results = []
best_model = None
best_model_name = None
best_rmse = float("inf")

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results.append({
        "Model": model_name,
        "RMSE": rmse,
        "MAE": mae,
        "R²": r2
    })

    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_model_name = model_name

    print(f"Model: {model_name}")
    print(f"  RMSE: {rmse}")
    print(f"  MAE: {mae}")
    print(f"  R²: {r2}\n")

# -----------------------
#  Step 5: Save Best Model Locally
print(f"\n Best model: {best_model_name} (RMSE = {best_rmse:.2f})")
joblib.dump(best_model, "best_model.pkl")

S3_KEY = "models/best_model.pkl"  # Example: "models/best_model.pkl"

# Upload
s3.upload_file("best_model.pkl", bucket_name, S3_KEY)
print(f"Model uploaded to s3://{bucket_name}/{S3_KEY}")

# -----------------------
# Step 7: Summary
results_df = pd.DataFrame(results)
print("\n Summary of Model Performance:")
print(results_df)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Model: Random_Forest
  RMSE: 0.040923085449048845
  MAE: 0.005547578472924301
  R²: 0.9980067679013092



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Model: Gradient_Boosting
  RMSE: 0.04902371832856705
  MAE: 0.0162210659871067
  R²: 0.9971395548259236

Model: Linear_Regression
  RMSE: 0.5862036785894914
  MAE: 0.4699046158963499
  R²: 0.5910048007922956

Model: Ridge_Regression
  RMSE: 0.5886032175553206
  MAE: 0.47329752761990995
  R²: 0.5876496238743758



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Model: Support_Vector_Regressor
  RMSE: 0.4243107366342194
  MAE: 0.29347555755154797
  R²: 0.7857162816936925



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Model: Neural_Network
  RMSE: 2.3819400258312684
  MAE: 1.7966987710908204
  R²: -5.752782721860919


 Best model: Random_Forest (RMSE = 0.04)


Model uploaded to s3://my-feature-store-data/models/best_model.pkl

 Summary of Model Performance:
                      Model      RMSE       MAE        R²
0             Random_Forest  0.040923  0.005548  0.998007
1         Gradient_Boosting  0.049024  0.016221  0.997140
2         Linear_Regression  0.586204  0.469905  0.591005
3          Ridge_Regression  0.588603  0.473298  0.587650
4  Support_Vector_Regressor  0.424311  0.293476  0.785716
5            Neural_Network  2.381940  1.796699 -5.752783
